#Mount drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install libraries

In [2]:
!pip install -q python-dotenv langchain langchain-openai openai langchain-community chromadb langchainhub gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

# Parameters

In [10]:
selected_model = "gpt-3.5-turbo"
temperature = 0.4
top_p = 0.8

#API key

In [3]:
import os
from dotenv import load_dotenv
load_dotenv("drive/MyDrive/Colab/.env")

True

# Connecting to DB with embeddings

In [5]:
import chromadb
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# Path where embeddings are stored
CHROMA_DATA_PATH = 'drive/MyDrive/Colab/docs/houdini_embeddings/'
COLLECTION_NAME = "emb"

# Initialize ChromaDB client
client = chromadb.PersistentClient(path=CHROMA_DATA_PATH)

loaded_db = Chroma(persist_directory=CHROMA_DATA_PATH, embedding_function=OpenAIEmbeddings())

# RAG pipeline

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub

In [14]:
llm = ChatOpenAI(model=selected_model, temperature=temperature, model_kwargs={'top_p': top_p})
prompt = hub.pull("rlm/rag-prompt")

## Overwrite prompt template

In [15]:
prompt.messages[0].prompt.template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Firstly provide a short summarized answer. Afterwards provide a detailed answer.\nQuestion: {question} \nContext: {context} \nAnswer:"""

# QA chain

In [16]:
qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=loaded_db.as_retriever(),
        chain_type_kwargs={"prompt": prompt}
    )

In [17]:
question = "How to create velocity field from geometry in houdini?"
result = qa_chain.invoke({"query": question})
result

{'query': 'How to create velocity field from geometry in houdini?',
 'result': "To create a velocity field from geometry in Houdini, you can use the 'vdb from polygons' node. This node can convert meshes into SDF or fog and create a velocity field if your mesh has a @v attribute. You can create a swirled @v in a wrangle with @v = cross(@N,{0,1,0}) on your mesh, then use 'vdb from polygons' to make a density field and create a @vel field from @v. If you need to transfer the @v attribute from a curve to @vel of the volume, you can use a volume wrangle to achieve this.\n\nDetailed Answer:\nTo create a velocity field from geometry in Houdini, you can follow these steps:\n\n1. Start by using the 'vdb from polygons' node. This node is useful for converting meshes into signed distance fields (SDF) or fog volumes. It can also create a velocity field if your mesh has a @v attribute.\n\n2. If your mesh doesn't have a @v attribute, you can create one in a wrangle node. For example, you can create

# Adding UI

In [21]:
import gradio as gr

# Initialize a global list to keep track of conversation history
conversation_history = []


def talk(message, history):
    # The 'history' parameter includes past interactions
    # We'll extract the question and answer texts to build a full context
    context = " ".join([f"Q: {exchange['input']} A: {exchange['output']}" for exchange in history if 'input' in exchange and 'output' in exchange])
    full_question = f"{context} Q: {message}"

    # Assume 'qa_chain' is your model that processes the full question
    result = qa_chain.invoke({"query": full_question})
    answer = result['result']

    return answer

# Define the chatbot interface layout and properties
TITLE = "Houdini Chatbot"
DESCRIPTION = f"{selected_model} with RAG to answer Houdini-related questions based on tokeru.com/cgwiki"
chatbot = gr.Chatbot(
    show_label=True,
    show_share_button=True,
    show_copy_button=True,
    likeable=True,
    layout="bubble",
    bubble_full_width=True,
    height=700,
)

# Create the ChatInterface
demo = gr.ChatInterface(
    fn=talk,
    chatbot=chatbot,
    theme="Soft",
    title=TITLE,
    description=DESCRIPTION
)

In [22]:
# Launch the Gradio interface
demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://da02eeedd5894274bd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://da02eeedd5894274bd.gradio.live
